In [ ]:
import pandas as pd
df = pd.read_csv("tiktok_hashtags.csv",delimiter=";")
df.drop(columns=['url'], inplace=True)
df = df[df['hashtags'].str.contains('Error') == False]
df = df[df['hashtags'].notna()] 

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# 🔹 Ensure 'hashtags' column is in list format
df['hashtags'] = df['hashtags'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# 🔹 Manually Labeled Data to increase precision of the predictions 
positive_hashtags = {'#happy', '#love', '#cat', '#curly', '#baby', '#kitten', '#positivity','#yalandünya','#funny','#cute','#comedy','#yemek','#kindness','#toddler','#sisters','#music','#müzik','#modernfamily','#housedesign','#outfit','#motivasyon','#friends'}
negative_hashtags = {'#bojackhorseman', '#fleabag', '#overthinking','#quote', '#midterm','#final','#relatable','#university','#breakup','#3amthoughts','#nostalji','#thoughtdaughter','#sad','#scary','#psikoloji','#büyükevablukada','#therapy'}

# 🔹 Function to classify hashtags
def categorize_hashtags(hashtags):
    hashtags = set(hashtags)  # Convert list to set
    if hashtags & positive_hashtags:
        return 'positive'
    elif hashtags & negative_hashtags:
        return 'negative'
    else:
        return 'neutral'  # Exclude neutral labels from training

# 🔹 Apply classification
df['sentiment'] = df['hashtags'].apply(categorize_hashtags)

# 🔹 Remove neutral rows
df = df[df['sentiment'] != 'neutral']

# 🔹 Convert list of hashtags into a single string
df['hashtag_text'] = df['hashtags'].apply(lambda x: ' '.join(x))

# 🔹 Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(df['hashtag_text'], df['sentiment'], test_size=0.2, random_state=42)

# 🔹 Convert text to numerical format using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 🔹 Train an SVM classifier
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_vec, y_train)

# 🔹 Make predictions
y_pred = svm_model.predict(X_test_vec)

# 🔹 Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ SVM Model Accuracy: {accuracy:.2f}")

# 🔹 Predict for new hashtags
new_hashtags = ["#dog #funny #music", "#bojackhorseman #sad", "#positivity #meditation"]
new_vec = vectorizer.transform(new_hashtags)
new_predictions = svm_model.predict(new_vec)

# 🔹 Add predicted sentiment as a new column
df['predicted_sentiment'] = svm_model.predict(vectorizer.transform(df['hashtag_text']))

# 🔹 Display final dataset with only predictions and time
df.drop(columns=['hashtags'], inplace=True)
df.drop(columns=['sentiment'], inplace=True)
df.drop(columns=['hashtag_text'], inplace=True)
df.head()


✅ SVM Model Accuracy: 0.83


,date,predicted_sentiment
10,2024-11-28 00:09:38,positive
18,2024-11-27 11:31:45,positive
26,2024-11-25 18:26:58,negative
56,2024-11-20 01:45:36,positive
61,2024-11-20 01:27:48,positive
